In [14]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
load_dotenv()
class LLMFactory:
    @staticmethod
    def create_llm() -> ChatOpenAI:
        return ChatOpenAI(
            temperature=0,
            model="deepseek-chat",
            api_key=os.getenv('DEEPSEEK_API_KEY'),
            base_url=os.getenv('DEEPSEEK_BASE_URL')
        )    

String Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("你是一个{name}，请帮我起一个具有{country}特色的{sex}名字")
prompt.format(name="算命大师", country="中国", sex="女孩")



123


Chat Prompt Teplate

In [2]:
from langchain.prompts import ChatPromptTemplate

ChatPromptTemplate.from_messages([
    ("system", "你是一个起名大师，你的名字叫{name}"),
    ("human", "你好{name}, 你感觉如何"),
    ("ai", "我很好，谢谢"),
    ("human", "{user_input}")
]).format_messages(name="算命大师", user_input="请帮我起一个女孩名字")

[SystemMessage(content='你是一个起名大师，你的名字叫算命大师', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好算命大师, 你感觉如何', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我很好，谢谢', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请帮我起一个女孩名字', additional_kwargs={}, response_metadata={})]

In [4]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

sy = SystemMessage(content="你是一个起名大师", additional_kwargs={"大师姓名": "陈瞎子"})

hu = HumanMessage(content="请问大师叫什么")

ai = AIMessage(content="我叫陈瞎子")


[sy, hu, ai]


[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}, response_metadata={}),
 HumanMessage(content='请问大师叫什么', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我叫陈瞎子', additional_kwargs={}, response_metadata={})]

In [8]:
from langchain.prompts import AIMessagePromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatMessagePromptTemplate

prompt = "愿{subject}与你同在!"
#支持自定义角色
chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者", template=prompt)
chat_message_prompt.format(subject="原力")

systemp_message_prompt = SystemMessagePromptTemplate.from_template(role="天行者", template=prompt)
systemp_message_prompt.format(subject="原力")

SystemMessage(content='愿原力与你同在!', additional_kwargs={}, response_metadata={})

构建自己的自定义模版

In [ ]:
from langchain.prompts import StringPromptTemplate

def hello_word() -> str:
    return f"Hello world"

PROMPT = """
你是一名非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：{function_name}
源代码：
{source_code}
代码解释：
"""
import inspect

def get_source_code(func_name: str) -> str:
    return inspect.getsource(func_name)
# 自定义模版class
class MyPrompt(StringPromptTemplate):
    def format(self, **kwargs: str) -> str:
        function_name = kwargs.get("function_name")
        source_code = get_source_code(function_name)
        # 生成提示词模版
        return PROMPT.format(function_name=function_name.__name__, source_code=source_code)
    
    
re = MyPrompt(input_variables=["function_name"]).format(function_name=hello_word)
print(re)

llm = LLMFactory.create_llm()
llm.predict(re)


你是一名非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称：hello_word
源代码：
def hello_word() -> str:
    return f"Hello word"

代码解释：



C:\Users\longyu\AppData\Local\Temp\ipykernel_2056\3601633994.py:30: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict(re)


'函数名称：hello_word  \n\n源代码：  \n```python\ndef hello_word() -> str:\n    return f"Hello word"\n```  \n\n代码解释：  \n1. **功能**：定义一个名为 `hello_word` 的函数，调用时会返回字符串 `"Hello word"`。  \n2. **返回值类型**：通过 `-> str` 显式声明返回值为字符串类型（类型注解）。  \n3. **实现细节**：直接使用 f-string（格式化字符串）返回固定内容，无参数或复杂逻辑。  \n4. **用途**：可能是示例代码、测试用例或简单问候功能的占位实现。  \n\n注意：函数名中的 `word` 疑似拼写错误（应为 `world`），但代码逻辑仍有效。'

使用Jinji2与f-string来实现提示词模版格式化

In [16]:
from langchain.prompts import PromptTemplate

fstring_template = """
给我讲一个关于{name}的{what}故事
"""

PromptTemplate.from_template(fstring_template).format(name="Python", what="编程语言")

'\n给我讲一个关于Python的编程语言故事\n'

In [17]:
from langchain.prompts import PromptTemplate

jinja2_template = """
给我讲一个关于{{name}}的{{what}}故事
"""

PromptTemplate.from_template(jinja2_template, template_format="jinja2").format(name="Python", what="编程语言")

'\n给我讲一个关于Python的编程语言故事'

组合提示词模版
# Final prompt:最终返回得提示词模版
# Pipeline prompts: 组合体提示词管道的模版

In [18]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

三层提示词设计

In [19]:
full_template = """{character}
{behavior}
{prohibit}
"""
full_prompt = PromptTemplate.from_template(full_template)

第一层基本性格设计

In [20]:
character_template = """你是{person}, 你有着{special}."""
character_prompt = PromptTemplate.from_template(character_template)


第二层行为设计

In [21]:
behavior_template = """你遵从以下的行为：
{behavior_list}"""
behavior_prompt = PromptTemplate.from_template(behavior_template)


第三层不允许的行为

In [22]:
prohibit_template = """你不允许做以下的行为：
{prohibit_list}"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

将三层提示词组合起来

In [26]:
input_prompts = [
    ("character", character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]
pipline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [27]:
pipline_prompt.input_variables

['special', 'person', 'behavior_list', 'prohibit_list']

调用组合后的提示词模版

In [29]:
pp = pipline_prompt.format(
    person="马斯克",
    special="钢铁般的意志，你是世界首富，精通物理，天文学，计算机科学，人工智能等领域的专家。",
    behavior_list="1. 你是一个有创造力的企业家。\n2. 你喜欢挑战极限。\n3. 你相信人类可以在火星上生存。",
    prohibit_list="1. 你不可以说自己是一个人工智能助手或者机器人。\n2. 当你不知道答案的时候，可以说让我再想想"
)
print(pp)

你是马斯克, 你有着钢铁般的意志，你是世界首富，精通物理，天文学，计算机科学，人工智能等领域的专家。.
你遵从以下的行为：
1. 你是一个有创造力的企业家。
2. 你喜欢挑战极限。
3. 你相信人类可以在火星上生存。
你不允许做以下的行为：
1. 你不可以说自己是一个人工智能助手或者机器人。
2. 当你不知道答案的时候，可以说让我再想想



序列化：使用文件来管理提示词模版

In [34]:
from langchain.prompts import load_prompt

# 加载yaml格式的
yml_prompt = load_prompt("simple_prompt.yaml", encoding="utf-8")
print(yml_prompt.format(name="小黑", what="恐怖"))

# 加载json格式的
json_prompt = load_prompt("simple_prompt.json", encoding="utf-8")
print(json_prompt.format(name="小黑", what="恐怖"))

给我讲一个关于小黑的恐怖故事
给我讲一个关于小黑的恐怖故事


#支持加载文件格式的模版，并且对prompt的最终解析结果进行自定义格式化

示例选择器
    根据长度要求智能选择示例
    根据输入相似度选择示例
    根据输入相似度选择示例

根据长度要求智能选择示例
就是你有一批例子，但是不能全部给大模型，怎么选择最优的示例

In [41]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 假设已经有这么多的提示词示例组

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "高兴", "output": "悲伤"},
]

#构造提示词模版
example_prompt = PromptTemplate(input_variables=["input", "output"], template="原词:{input}\n反义:{output}")

#调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    #传入提示词模版
    examples=examples,
    #传入提示词模版
    example_prompt=example_prompt,
    #设置格式化后的提示词最大长度
    max_length=25,
    # 内置的get_text_length,如果默认分词计算方式不满足，可以自己扩展
    #get_txt_length:Callable[[str], int] = lambda x: len(re.split("\n| ", x))
    
)

#使用小样本提示词模版来实现动态示例的调用

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词:{adjective}\n反义:",
    input_variables=["adjective"]
)

In [42]:
print(dynamic_prompt.format(adjective="big"))

给出每个输入词的反义词

原词:happy
反义:sad

原词:tall
反义:short

原词:sunny
反义:gloomy

原词:windy
反义:calm

原词:高兴
反义:悲伤

原词:big
反义:


In [44]:
#如果输入长度很长，则最终输出会根据长度要求减少
long_string = "big and huge and massive and gigantic and tall and much much much bigger then everone"
print(dynamic_prompt.format(adjective=long_string))

给出每个输入词的反义词

原词:happy
反义:sad

原词:tall
反义:short

原词:sunny
反义:gloomy

原词:windy
反义:calm

原词:big and huge and massive and gigantic and tall and much much much bigger then everone
反义:


根据输入相似度选择示例(最大边际相关性)
    #MMR是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡
    #MMR会首先找出与输入最相似（即余玄相似度最大）的样本
    -然后在迭代添加样本的过程中，对于已选择样本过于接近（即相似度过高）的样本进行惩罚
    #MMR既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性
    #关注如何在相关性和多样性之间找到一个平衡

需要安装两个包， 一个tiktoken  一个faiss-cpu

In [ ]:
# 使用MMR来检索相关示例，以示例尽量符合输入
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate


import os 

#构造提示词模版

example_prompt = PromptTemplate(input_variables=["input", "output"], template="原词:{input}\n反义:{output}")
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    #传入示例组
    examples,
    #使用openai的嵌入来做相似性的搜索, 这里需要配置
    OpenAIEmbeddings(),
    #设置使用的向量数据库是什么
    FAISS,
    #结果条数
    k=2
)
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词:{adjective}\n反义:",
    input_variables=["adjective"]
)


In [ ]:
# 当我们输入一个描述情绪的词语，应该选择同样是描述情绪的一对示例组来填充提示词模版
print(mmr_prompt.format(adjective="难过"))

根据输入相似度选择示例（最大余玄相似度）
    ·通过计算两个向量之间的余玄值来衡量他们的相似度
    ·余玄值接近1，表示两个向量越相似
    ·主要关注如何准确衡量两个向量相似度
需要安装 chromadb

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
example_selector = SemanticSimilarityExampleSelector.from_examples(
    #传入示例组
    examples,
    #使用openai的嵌入来做相似性的搜索, 这里需要配置
    OpenAIEmbeddings(),
    #设置使用的向量数据库是什么
    Chroma,
    #结果条数
    k=1
)
sse_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词:{adjective}\n反义:",
    input_variables=["adjective"]
)

In [ ]:
from langchain.callbacks import get_openai_callback
# 使用回调来获取token使用情况和成本
with get_openai_callback() as cb:
    LLMFactory.create_llm().predict(sse_prompt.format(adjective="难过"))
    print(sse_prompt.format(adjective="难过"))
    print(f"Total tokens used: {cb.total_tokens}")
    print(f"Total cost: ${cb.total_cost:.6f}")
